In [ ]:
%matplotlib inline

import cv2
import numpy as np
import scipy

import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
data_ix = 4

In [ ]:
if data_ix == 0:
    img_list = [cv2.imread("dataset/data_image_stitching/im1.png"),
                cv2.imread("dataset/data_image_stitching/im2.png")]

elif data_ix == 1:
    img_list = [cv2.imread("dataset/data_image_stitching/im22.jpg"),
                cv2.imread("dataset/data_image_stitching/im23.jpg")]

elif data_ix == 2:
    img_list = [cv2.imread("output/stitched_img_1.png"),
                cv2.imread("dataset/data_image_stitching/im24.jpg")]

elif data_ix == 3:
    img_list = [cv2.imread("output/stitched_img_2.png"),
                cv2.imread("dataset/data_image_stitching/im25.jpg")]

elif data_ix == 4:
    img_list = [cv2.imread("dataset/data_image_stitching/im89.jpg"),
                cv2.imread("dataset/data_image_stitching/im90.jpg")]

In [ ]:
fig = plt.figure(figsize=(8*len(img_list), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_list)):
    plt.subplot(1, len(img_list), i+1)
    plt.imshow(cv2.cvtColor(img_list[i], cv2.COLOR_BGR2RGB))

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

keypoints = []
descriptors = []
img_keypoints = []

for img in img_list:
    cur_keypoints, cur_descriptors = sift.detectAndCompute(img, None)
    keypoints.append(cur_keypoints)
    descriptors.append(cur_descriptors)
    img_keypoints.append(cv2.drawKeypoints(img, cur_keypoints, None))

fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
def normalize_descriptor(descriptor):
    return (descriptor - np.min(descriptor)) / (np.max(descriptor) - np.min(descriptor))

In [ ]:
number_of_matches = 100
pairwise_distances = scipy.spatial.distance.cdist(normalize_descriptor(descriptors[0]), normalize_descriptor(descriptors[1]))
matched_points = []
for i in range(number_of_matches):
    min_index = np.argmin(pairwise_distances)
    first_point = min_index // pairwise_distances.shape[1]
    second_point = min_index % pairwise_distances.shape[1]
    matched_points.append((first_point, second_point))
    pairwise_distances[min_index // pairwise_distances.shape[1],:] = np.inf
    pairwise_distances[:,min_index % pairwise_distances.shape[1]] = np.inf

In [ ]:
filtered_keypoints = []
filtered_keypoints.append([keypoints[0][match[0]] for match in matched_points])
filtered_keypoints.append([keypoints[1][match[1]] for match in matched_points])
img_keypoints = []
img_keypoints.append(cv2.drawKeypoints(img_list[0], filtered_keypoints[0], None))
img_keypoints.append(cv2.drawKeypoints(img_list[1], filtered_keypoints[1], None))

fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
matches1to2 = [cv2.DMatch(i, i, 0) for i in range(number_of_matches)]
matching_img = cv2.drawMatches(img_list[0], filtered_keypoints[0], img_list[1], filtered_keypoints[1], matches1to2, None)
fig = plt.figure(figsize=(16, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(matching_img, cv2.COLOR_BGR2RGB))

In [ ]:
(homography, mask) = cv2.findHomography(np.float32([kp.pt for kp in filtered_keypoints[1]]), 
                                        np.float32([kp.pt for kp in filtered_keypoints[0]]), 
                                        cv2.RANSAC, ransacReprojThreshold=3.0)

height1, width1 = img_list[0].shape[:2]
height2, width2 = img_list[1].shape[:2]
pts = np.concatenate((np.float32([[0, 0], [0, height1], [width1, height1], [width1, 0]]).reshape(-1, 1, 2), 
                      cv2.perspectiveTransform(np.float32([[0, 0], [0, height2], [width2, height2], [width2, 0]]).reshape(-1, 1, 2), homography)), 
                     axis=0)
[xmin, ymin] = np.int32(np.min(pts, axis=0).squeeze() - 0.5)
[xmax, ymax] = np.int32(np.max(pts, axis=0).squeeze() + 0.5)
t = [-xmin, -ymin]
ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])

stitch_r = np.zeros((ymax - ymin, xmax - xmin, 3)).astype(np.uint8)
trans_mat = cv2.invert(ht.dot(homography))[1]
for i in range(ymax - ymin):
    for j in range(xmax - xmin):
        img_i = int((trans_mat[1][0] * j + trans_mat[1][1] * i + trans_mat[1][2]) / (trans_mat[2][0] * j + trans_mat[2][1] * i + trans_mat[2][2]))
        img_j = int((trans_mat[0][0] * j + trans_mat[0][1] * i + trans_mat[0][2]) / (trans_mat[2][0] * j + trans_mat[2][1] * i + trans_mat[2][2]))
        if img_i >= 0 and img_j >= 0 and img_i < img_list[1].shape[0] and img_j < img_list[1].shape[1]:
            stitch_r[i][j] = img_list[1][img_i][img_j]

In [ ]:
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(stitch_r, cv2.COLOR_BGR2RGB))

In [ ]:
stitch_l = np.zeros_like(stitch_r)
stitch_l[t[1]:height1 + t[1], t[0]:width1 + t[0]] = img_list[0]
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(stitch_l, cv2.COLOR_BGR2RGB))

In [ ]:
stitch_mask = np.ones_like(stitch_r, dtype=np.float32)
stitch_mask[stitch_r > 0] = 0
stitch_mask[stitch_l == 0] = 1
start_col = np.min(np.where(stitch_mask == 0)[1])
end_col = np.max(np.where(stitch_mask == 0)[1])
for i in range(stitch_mask.shape[1]):
    col_value = np.ones(stitch_mask[:,i].shape) * (i - start_col) / (end_col - start_col)
    col_value[np.logical_and(stitch_r[:,i] > 0, stitch_l[:,i] == 0)] = 1
    col_value[np.logical_and(stitch_r[:,i] == 0, stitch_l[:,i] > 0)] = 0
    col_value = np.clip(col_value, 0.0, 1.0)
    stitch_mask[:,i] = col_value
result = np.uint8((stitch_r * stitch_mask) + (stitch_l * (1 - stitch_mask)))
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

In [ ]:
cv2.imwrite(f"output/stitched_img_{data_ix}.png", result)